# ELG5381 Final Project: (D)DQN for RWA 

In [1]:
import sys,os,pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output

path ='C:/Users/D-GAMING-MACHINE/Documents/ELG5381/BranchingDQN/'
sys.path.append(path)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'


Num GPUs Available:  1


# Check if Nvidia GPU is Available

In [2]:
# silencing tensorflow warnings
import tensorflow as tf
import gymnasium as gym
import logging
logging.getLogger('tensorflow').setLevel(logging.FATAL)
tf.ConfigProto(device_count = {'GPU': 1})
tf.__version__

'2.9.1'

# Import Stable-Baseline3

In [17]:
import stable_baselines3
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common import results_plotter
stable_baselines3.__version__

In [18]:
load = 50  # Traffic load, measured in Erlangs
seed = 15  # Seed of environment
episodes = 100  # Number of episodes per execution
episode_length = 1000  # Episode Length
k_paths = 5 # k parameter  for Sk-P
num_layers = 2
layer_size = 128
regu_scalar = 1e-4
gamma = 0.95
batch_size = 16  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 100  # @param {type:"integer"}

# Create log dir
log_dir = "./tmp/RWA-dqn/"
os.makedirs(log_dir, exist_ok=True)

# loading the topology binary file containing the graph and the k-shortest paths
# if you want to generate your own binary topology file, check examples/create_topology_rmsa.py
topology_name = 'nsfnet_chen'

with open(f'topologies/{topology_name}_{k_paths}-paths_6-modulations.h5', 'rb') as f:
    topology = pickle.load(f)

monitor_info_keywords=('episode_service_blocking_rate','episode_bit_rate_blocking_rate')

# node probabilities from https://github.com/xiaoliangchenUCD/DeepRMSA/blob/6708e9a023df1ec05bfdc77804b6829e33cacfe4/Deep_RMSA_A3C.py#L77
node_request_probabilities = np.ones(14)*13*2/13/14

# Environment arguments for the simulation
env_args = dict(topology=topology, seed=seed, allow_rejection=True, load=load, 
               episode_length=episode_length)

env_name = 'RWA-v0'

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

In [19]:
topology.edges()

EdgeView([('1', '2'), ('1', '3'), ('1', '8'), ('2', '3'), ('2', '4'), ('3', '6'), ('4', '5'), ('4', '11'), ('5', '6'), ('5', '7'), ('6', '10'), ('6', '14'), ('7', '8'), ('7', '10'), ('8', '9'), ('9', '10'), ('9', '12'), ('9', '13'), ('11', '12'), ('11', '13'), ('12', '14'), ('13', '14')])

In [20]:
env_rnd = gym.make(env_name, **env_args)
# evaluating the environment that acts completely random both for path and wavelength
mean_reward_rnd, std_reward_rnd = evaluate_heuristic(
    env_rnd, random_policy, n_eval_episodes=episodes
)
# env_rnd.render()  # uncomment to visualize a representation of the environment
print("\nRnd:", mean_reward_rnd, std_reward_rnd)
rnd_path_action_probability = np.sum(env_rnd.actions_output, axis=1) / np.sum(
    env_rnd.actions_output
)
rnd_wavelength_action_probability = np.sum(env_rnd.actions_output, axis=0) / np.sum(
    env_rnd.actions_output
)
print(
    "\tPath action probability:",
    np.sum(env_rnd.actions_output, axis=1) / np.sum(env_rnd.actions_output),
)
# print('Wavelength action probability:', np.sum(env_rnd.actions_output, axis=0) / np.sum(env_rnd.actions_output))


Rnd: 654.11 12.461055332514979
	Path action probability: [0.168   0.16675 0.16566 0.16711 0.16576 0.16672]


In [22]:
fc_layer_params = (100, 50)
rwa_env = gym.make(env_name, **env_args)
action_tensor_spec = tensor_spec.from_spec(rwa_env.action_space)
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

ValueError: No known conversion from type `<class 'gym.spaces.multi_discrete.MultiDiscrete'>` to a TensorSpec.  Saw:
  MultiDiscrete([ 6 81])

In [7]:
rwa_env = tf_py_environment.TFPyEnvironment(rwa_env)
opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

q_net = q_network.QNetwork(
  rwa_env.observation_space(),
  rwa_env.action_space,
  fc_layer_params=(100,))

agent = dqn_agent.DdqnAgent(
  rwa_env.time_step_spec(),
  rwa_env.action_spec(),
  gamma = gamma,
  q_network=q_net,
  optimizer=opt,
  td_errors_loss_fn=common.element_wise_squared_loss,
  train_step_counter=tf.Variable(0))

agent.initialize()

AttributeError: 'RWAEnv' object has no attribute 'observation_spec'

In [ ]:
a = agent.learn(total_timesteps=10000, callback=callback)

In [ ]:
results_plotter.plot_results([log_dir], 1e4, results_plotter.X_TIMESTEPS, "DeepRMSA PPO")